In [1]:
# PYTHON CODE
# Module imports

import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

from IPython.core.display import display, HTML

<h1>RF Tools Demo </h1>
    
Demo for the rf_tools octave package

ISMRM 2019, Montreal Canada

Run this with

    jupyter notebook rf_tools_demo.ipynb
    
This requires that octave be installed, the signal toolbox, and the octave kernel for jupyter.

<h2> Outline </h2>

There are four parts to this tutorial
- RF Pulse simulator -- abr.m
- Shinnar-Le Roux pulse design routines -- b2rf.m, and friends
- Minimum and maximum phase pulses -- dzmp.m
- 2D spiral pulse design -- dz2d.m

The first three sections cover the Shinnar-Le Roux (SLR) pulse design method described in

"Parameter relations for the Shinnar-Le Roux selective excitation pulse design algorithm," J Pauly, P Le Roux, D Nishimura, A Macovski IEEE transactions on medical imaging 10 (1), 53-65

First we need to load the signal toolbox

In [2]:
pkg load signal

<h2> RF Pulse Simulator </h2>

rf_tools has an RF pulse simulator that computes the rotations produced by
your rf pulses.

It does this using spinors, and the Cayley-Klein parameters $\alpha$ and $\beta$.
These two complex numbers completely define a rotation.

Once the rotation has been computed, any RF profile can be computed.

First, we need an RF pulse

In [3]:
rf = msinc(256,2);

This is just a Hamming windowed sinc.

Normalize to $\pi/2$, and plot it.  cplot() takes a complex signal and plots both the real and imaginary parts.

In [4]:
rf = rf*(pi/2)/sum(rf);

re_rf = real(rf);
im_rf = imag(rf);

In [5]:
%get re_rf --from Octave
%get im_rf --from Octave

re = go.Scatter(
    x = [i for i in range(len(re_rf))],
    y = re_rf,
    name = 'Real',
    text = 'Real',
    hoverinfo = 'x+y+text', 
    line = dict(
        color = ('rgb(22, 96, 167)'),
        ),
)

im = go.Scatter(
    x = [i for i in range(len(im_rf))],
    y = im_rf,
    name = 'Imaginary',
    text = 'Imaginary',
    hoverinfo = 'x+y+text',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        ),
)

data = [re, im]

layout = go.Layout(
    width=600,
    height=350,
    margin=go.layout.Margin(
        l=100,
        r=50,
        b=60,
        t=0,
    ),
    xaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.65,
        y=0.55,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    )
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

Next we need a vector of spatial locations.  By default the 
simulator assumes that the gradient area integrates to 2 pi over 
the pulse, so the spatial locations are multiples of 2 pi.

In [6]:
x = [-64:64]/4;

Then we simulate the result with abr(rf,x).  This stands for "alpha-beta rotator",

In [7]:
[a b] = abr(rf,x);

re_a = real(a);
im_a = imag(a);

re_b = real(b);
im_b = imag(b);

The $\alpha$ slice profile mostly characterizes the phase of the profile.

In [8]:
%get re_a --from Octave
%get im_a --from Octave

wm = go.Scatter(
    x = [i for i in range(len(re_a))],
    y = re_a,
    name = 'Real',
    text = 'Real',
    hoverinfo = 'x+y+text',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        ),
)

gm = go.Scatter(
    x = [i for i in range(len(im_a))],
    y = im_a,
    name = 'Imaginary',
    text = 'Imaginary',
    hoverinfo = 'x+y+text',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        ),
)



data = [wm, gm]

layout = go.Layout(
    width=600,
    height=350,
    margin=go.layout.Margin(
        l=100,
        r=50,
        b=60,
        t=0,
    ),
    xaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    annotations=[
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='Alpha',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    legend=dict(
        x=0.385,
        y=1.10,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    )
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

The $\beta$ profile looks like the slice profile we expect, and has an amplitude of $\sin(\phi/2)$ if $\phi$ is the flip angle.  This is a $\pi/2$ pulse so the amplitude is $\sin(\pi/4) = \sqrt{2}/{2}$, or about 0.707.  It is negative because we are using right hand rotations, while protons rotate in the negative sense,

In [9]:
%get re_b --from Octave
%get im_b --from Octave

re = go.Scatter(
    x = [i for i in range(len(re_b))],
    y = re_b,
    name = 'Real',
    text = 'Real',
    hoverinfo = 'x+y+text',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        ),
)

im = go.Scatter(
    x = [i for i in range(len(im_b))],
    y = im_b,
    name = 'Imaginary',
    text = 'Imaginary',
    hoverinfo = 'x+y+text',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        ),
)



data = [re, im]

layout = go.Layout(
    width=600,
    height=350,
    margin=go.layout.Margin(
        l=100,
        r=50,
        b=60,
        t=0,
    ),
    xaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    annotations=[
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='Beta',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    legend=dict(
        x=0.70,
        y=0.55,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    )
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

This is the $\beta$ slice profile. $\alpha$ and $\beta$ determine the rotation produced by the RF pulse for any initial magnetization.  We can then compute any slice profile we'd like.

The excitation profile is $M_{xy} = 2 \alpha^*\beta$, and this is computed by ab2ex(a,b),

In [10]:
mxy_ex = ab2ex(a,b);

mxy_ex_abs = abs(mxy_ex);

In [11]:
%get x --from Octave
%get mxy_ex_abs --from Octave

mag = go.Scatter(
    x = [i for i in range(len(mxy_ex_abs))],
    y = mxy_ex_abs,
    name = 'Slice profile',
    text = 'Slice profile',
    hoverinfo = 'x+y+text',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        ),
)

data = [mag]

layout = go.Layout(
    width=600,
    height=350,
    margin=go.layout.Margin(
        l=100,
        r=50,
        b=60,
        t=40,
    ),
    xaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=1.15,
            showarrow=False,
            text='Slice profile',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
    ],
    legend=dict(
        x=0.70,
        y=0.55,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    )
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

The inversion profile is $M_z = 1-2\beta\beta^*$, and is computed by ab2inv(a,b),

In [12]:
mz_inv = ab2inv(a,b);

re_mz_inv = real(mz_inv);

In [13]:
%get x --from Octave
%get re_mz_inv --from Octave

mag = go.Scatter(
    x = [i for i in range(len(re_mz_inv))],
    y = re_mz_inv,
    name = 'Slice profile',
    text = 'Slice profile',
    hoverinfo = 'x+y+text',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        ),
)

data = [mag]

layout = go.Layout(
    width=600,
    height=350,
    margin=go.layout.Margin(
        l=100,
        r=50,
        b=60,
        t=40,
    ),
    xaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=1.15,
            showarrow=False,
            text='Inversion or Saturation Profile',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
    ],
    legend=dict(
        x=0.70,
        y=0.55,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    )
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

The spin echo profile is $M_{xy} =  -i \beta^2$, and is computed by ab2se(a,b),

In [14]:
mxy_se = ab2se(a,b);

re_mxy_se = real(mxy_se);

In [15]:
%get x --from Octave
%get re_mxy_se --from Octave

mag = go.Scatter(
    x = [i for i in range(len(re_mxy_se))],
    y = re_mxy_se,
    name = 'Slice profile',
    text = 'Slice profile',
    hoverinfo = 'x+y+text',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        ),
)

data = [mag]

layout = go.Layout(
    width=600,
    height=350,
    margin=go.layout.Margin(
        l=100,
        r=50,
        b=60,
        t=40,
    ),
    xaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=1.15,
            showarrow=False,
            text='Spin-Echo Profile',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
    ],
    legend=dict(
        x=0.70,
        y=0.55,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    )
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

Not too surprising, a $\pi/2$ pulse is not a great spin echo pulse.

Now we just need better RF pulses to simulate! That is next.

<h2> Shinnar-Le Roux Pulse design demo </h2>

The previous section showed how to simulate the profile of an RF pulse.
In this section we show how to design much better RF pulses.

The SLR design starts by designing the beta polynomial, which is a lowpass
function scalled to sine of half the flip angle.  For an inversion pulse, 
this is sin(pi/2) = 1;

We start with a windowed sinc again

In [16]:
b = msinc(256,2);

This is scaled to 1, which is what we want.  We then design a consistent
$\alpha$, and then solve for the RF pulse. $\alpha$is determined uniquely 
given that we want a minimum power pulse, so the result is

In [17]:
rf = b2rf(b);

We will compare this to the windowed sinc scaled to $\pi$,

In [18]:
rfw = b*pi;

Compute the two responses as inversions.

In [19]:
x = [-64:64]/4;
mz = ab2inv(abr(rf,x));
mzw = ab2inv(abr(rfw,x));

In [20]:
%get x --from Octave
%get mz --from Octave
%get mzw --from Octave

re = go.Scatter(
    x = x,
    y = mz,
    name = 'slr',
    text = 'slr',
    hoverinfo = 'x+y+text', 
    line = dict(
        color = ('rgb(22, 96, 167)'),
        ),
)

im = go.Scatter(
    x = x,
    y = mzw,
    name = 'windowed sinc',
    text = 'windowed sinc',
    hoverinfo = 'x+y+text',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        ),
)

data = [re, im]

layout = go.Layout(
    width=600,
    height=350,
    margin=go.layout.Margin(
        l=100,
        r=50,
        b=60,
        t=0,
    ),
    xaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    legend=dict(
        x=0.70,
        y=0.65,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    )
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

Note the sharper profile of the SLR pulse.  This comes at a cost if we
compare RF pulses

In [21]:
t = [1:256]/32;

rfscale_rf = real(rfscale(rf,8));
rfscale_rfw = real(rfscale(rfw,8));

In [22]:
%get t --from Octave
%get rfscale_rf --from Octave
%get rfscale_rfw --from Octave

re = go.Scatter(
    x = t,
    y = rfscale_rf,
    name = 'slr',
    text = 'slr',
    hoverinfo = 'x+y+text', 
    line = dict(
        color = ('rgb(22, 96, 167)'),
        ),
)

im = go.Scatter(
    x = t,
    y = rfscale_rfw,
    name = 'windowed sinc',
    text = 'windowed sinc',
    hoverinfo = 'x+y+text',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        ),
)

data = [re, im]

layout = go.Layout(
    width=600,
    height=350,
    margin=go.layout.Margin(
        l=100,
        r=50,
        b=60,
        t=0,
    ),
    xaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.18,
            showarrow=False,
            text='time, ms',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='amplitude, kHz',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    legend=dict(
        x=0.70,
        y=0.65,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    )
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

rfscale(rf,t) takes an RF pulse that sums to its flip angle, and a pulse 
duration in ms, and scales it to an RF amplitude in kHz.  

The SLR pulse has almost twice the peak amplitude. Note that the SLR pulse has closer zero spacing, even though it was based on the windowed sinc waveform.

There are lots of different types of RF pulses (excitaiton, inversion, spin-echo, etc) and lots of different types of filter designs to base them on.  rf_tools provides a wrapper for thses called dzrf(n,tb,'pulse_type','filter_type',d1, d2).  If you 

In [23]:
help dzrf

'dzrf' is a function from the file /home/jovyan/work/RF-Tools-in-Octave/rf_tools_octave/dzrf.m

   rf = dzrf(np,tb,ptype,ftype,d1,d2)

  Designs an rf pulse.  There are a lot of options, most of
  which have defaults.  For example, a reasonable 100 sample
  tb=4 spin-echo pulse can be designed with

   rf = dzrf(100,4,'se')

  Inputs are:
    np -- number of points.         (required)
    tb -- time-bandwidth product    (required)
    ptype -- pulse type.  Options are:
      st  -- small tip angle         (default)
      ex  -- pi/2 excitation pulse
      se  -- pi spin-echo pulse
      sat -- pi/2 saturation pulse
      inv -- inversion pulse
    ftype -- filter design method.  Options are:
      ms  -- Hamming windowed sinc (an msinc)
      pm  -- Parks-McClellan equal ripple
      ls  -- Least Squares           (default)
      min -- Minimum phase (factored pm)
      max -- Maximum phase (reversed min)
    d1 -- Passband ripple        (default = 0.01)
    d2 -- Stopband ripple      

For example, to design a very low ripple inversion pulse using an equal ripple Parks-McClellan design, 

In [24]:
rf = dzrf(256,4,'inv','pm',0.001,0.001);

re_rf = real(rf);
im_rf = imag(rf);

ab2inv_rf = ab2inv(abr(rf,x));

re_ab2inv_rf = real(ab2inv_rf);
im_ab2inv_rf = imag(ab2inv_rf);

Something to try:
- Look at how the RF pulses and profiles chance as the flip angle decreases
- Scale $\beta$ to $\sin(\pi*0.99/2)$, $\sin(\pi*0.95/2)$, and $\sin(\pi*0.9/2)$ using the msinc filter from above.

<h2> Minimum and Maximum Phase RF Pulses </h2>

The SLR designs so far have been linear phase:
- They refocus (almost) perfectly
- They can be used as spin echo pulses


If you don't care about phase, there are much more selective pulses:
- Saturation pulses
- Inversion pulses
- Slab select pulses

These can be almost twice as selective as linear phase pulses.

You can design a minimum phase pulse by first designing a mimimum
phase $\beta$, scaling it to $\sin(\phi/2)$, where $\phi$ is your flip angle.

The function dzmp(n,tb,d1,d2) designs a minimum phase waveform with n samples, a time-bandwdith product of 8, and a passband ripple of d1, and a stopband ripple of d2. These are equalripple designs, which can end up with spikes ("Conolly wings") at the ends. The output of dzmp is  scaled to 1, so this corresponds to an inversion pulse.

In [25]:
bm = dzmp(256,8,0.001,0.001);
rfm = b2rf(bm);
t = [1:256]/32;

re_rfscale_rfm = real(rfscale(rfm,8));

In [26]:
%get t --from Octave
%get re_rfscale_rfm --from Octave

mag = go.Scatter(
    x = t,
    y = re_rfscale_rfm,
    name = 'Slice profile',
    text = 'Slice profile',
    hoverinfo = 'x+y+text',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        ),
)

data = [mag]

layout = go.Layout(
    width=600,
    height=350,
    margin=go.layout.Margin(
        l=100,
        r=50,
        b=60,
        t=40,
    ),
    xaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.18,
            showarrow=False,
            text='time, ms',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='amplitude, kHz',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    legend=dict(
        x=0.70,
        y=0.55,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    )
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

The linear phase inversion with the same time bandwidth product from the pevious demos is

In [27]:
bl = msinc(256,2);
rfl = b2rf(bl);

re_rfscale_rfl = real(rfscale(rfl,8));

Comparing the two rf pulses

In [28]:
%get re_rfscale_rfl --from Octave

plot_1 = go.Scatter(
    x = t,
    y = re_rfscale_rfm,
    name = 'minimum phase',
    text = 'minimum phase',
    hoverinfo = 'x+y+text',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        ),
)

plot_2 = go.Scatter(
    x = t,
    y = re_rfscale_rfl,
    name = 'linear phase',
    text = 'linear phase',
    hoverinfo = 'x+y+text',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        ),
)

data = [plot_1, plot_2]

layout = go.Layout(
    width=600,
    height=350,
    margin=go.layout.Margin(
        l=100,
        r=50,
        b=60,
        t=40,
    ),
    xaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.18,
            showarrow=False,
            text='time, ms',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='amplitude, kHz',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    legend=dict(
        x=0.65,
        y=0.65,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    )
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

We can compare the profiles as

In [29]:
x = [-64:64]/4;
mzm = ab2inv(abr(rfm,x));
mzl = ab2inv(abr(rfl,x));

In [30]:
%get mzm --from Octave
%get mzl --from Octave

plot_1 = go.Scatter(
    x = x,
    y = mzm,
    name = 'minimum phase',
    text = 'minimum phase',
    hoverinfo = 'x+y+text',
    line = dict(
        color = ('rgb(22, 96, 167)'),
        ),
)

plot_2 = go.Scatter(
    x = x,
    y = mzl,
    name = 'linear phase',
    text = 'linear phase',
    hoverinfo = 'x+y+text',
    line = dict(
        color = ('rgb(205, 12, 24)'),
        ),
)

data = [plot_1, plot_2]

layout = go.Layout(
    width=600,
    height=350,
    margin=go.layout.Margin(
        l=100,
        r=50,
        b=60,
        t=40,
    ),
    xaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    yaxis=dict(
        showgrid=False,
        linecolor='black',
        linewidth=2
    ),
    annotations=[
        dict(
            x=0.5004254919715793,
            y=-0.18,
            showarrow=False,
            text='time, ms',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=-0.15,
            y=0.5,
            showarrow=False,
            text='amplitude, kHz',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            textangle=-90,
            xref='paper',
            yref='paper'
        ),
    ],
    legend=dict(
        x=0.65,
        y=0.65,
        traceorder='normal',
        font=dict(
            family='Times New Roman',
            size=12,
            color='#000'
        ),
        bordercolor='#000000',
        borderwidth=2
    )
)

fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-line', config = config)

Click and drag to zoom in between the the transition bands (time = 0 to 5 ms)

This can be useful for other pulses, such as slab select pulses

Something to try:
- Design a minimum phase excitation pulse (scale $\beta$ to $\sqrt{2}/2$)
- Compare the complex excitation profile of the min phase pulse, and a max phase pulse (min phase reversed in time).

<h2> 2D Spiral Excitation Pulses </h2>

2D spiral pulses can be designed for any flip angle using Fourier designs
The flip angle is the Fourier transform of the k-space weighting.  This was described in

"A k-space analysis of small-tip-angle excitation" J Pauly, D Nishimura, A Macovski
Journal of Magnetic Resonance (1969) 81 (1), 43-56

"A linear class of large-tip-angle selective excitation pulses", J Pauly, D Nishimura, A Macovski
Journal of Magnetic Resonance (1969) 82 (3), 571-587

An example design is an eight turn pulse with 1 G/cm gradient, and 
2 G/cm/ms slew rates (10 mT/m and 20 mT/m/s)

In [31]:
[rf g] = dz2d(8,1,4,512,1,2);

Gradient duration is  8.205 ms


where the second argument is the spatial bandwidth in cycles/cm and
the fourth argument is the space-bandwidth product (like TBW for 1D pulses).

The pulse ends up being 8.2 ms. We can plot the RF pulse and $G_x$ and $G_y$ gradients as

In [32]:
t = [1:512]*8.205/512;

Next we want to simulate the excitation profile.  At this point it is scaled
to 1 radian.  To increase it to $\pi/2$

In [33]:
rf = rf*pi/2;

The abr() simulator will take 2D pulses.  First define spatial vectors

In [34]:
x = [-32:32]/4;
y = [-32:32]/4;

mxy = ab2ex(abr(rf,g,x,y));

abs_mxy = abs(mxy);


In [35]:
%get x --from Octave
%get y --from Octave
%get abs_mxy --from Octave

XX, YY = np.meshgrid(x, y)
ZZ = np.asarray(abs_mxy)

lines = []
line_marker = dict(color='#0066FF', width=2)
for i, j, k in zip(XX, YY, ZZ):
    lines.append(go.Scatter3d(x=i, y=j, z=k, mode='lines', line=line_marker))

layout = go.Layout(
    showlegend=False
)

fig = go.Figure(data=lines, layout=layout)
iplot(fig, filename = 'basic-line', config = config)

This RF pulse is well refocused at the end of the pulse, as we can see by plotting $M_x$ and $-M_y$,

$M_x$ is only a couple of percent.

We can simulate over a broader spatial range to visualize the sidelobes which are inherent in this type of design

Note that the main lobe is imaginary, the first sidelobe is real, 
and the second sidelobe is imaginary.  Extra credit if you can explain 
this!

We can use the same pulse as a spin-echo pulse if we increase the flip 
angle to $\pi$, and make sure the gradients integrate to zero.  Here we do this with a single sample, in fact you'd want to add an additional gradient lobe, or incorportate the area by adjusting one of the crusher gradients.

In [36]:
rfse = rf*2;

rfse = [0; rf*2];
gse = [-sum(g); g];

mxyse = ab2se(abr(rfse,gse,x,y));

re_mxyse = real(mxyse);

In [37]:
%get x --from Octave
%get y --from Octave
%get re_mxyse --from Octave

XX, YY = np.meshgrid(x, y)
ZZ = np.asarray(re_mxyse)

lines = []
line_marker = dict(color='#0066FF', width=2)
for i, j, k in zip(XX, YY, ZZ):
    lines.append(go.Scatter3d(x=i, y=j, z=k, mode='lines', line=line_marker))

layout = go.Layout(
    annotations=[
        dict(
            x=0.5004254919715793,
            y=1.15,
            showarrow=False,
            text='My, Spin-echo Profile',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
    ],
    showlegend=False
)

fig = go.Figure(data=lines, layout=layout)
iplot(fig, filename = 'basic-line', config = config)

We've just plotted $M_y$ here, $M_x$ is again very small.

Things to try:
- Change the space-bandwidth product to 6 or 8
- Change the number of turns
- Do a two shot experiment, where g = -g for the second shot.  Add Mxy's.
- Use modern gradient numbers